In [17]:
import os
import sys
import django
from django.db import connection
import logging
import pandas as pd
from datetime import datetime

path = os.path.realpath('')

sys.path.insert(0, '/Users/victornguyen/Sites/07.book_management')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "book_management.settings")
django.setup()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger('User similarity calculator')

# Import Models
from title.models import Title, Author, Publisher
from main_site.models import Rating, Similarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
db_book_list = Title.objects.all()
db_book_list = [Title.book_info_as_dict(book) for book in db_book_list]
db_book_list = pd.DataFrame.from_dict(db_book_list, orient='columns')
db_book_list['type'] = 'database'
db_book_list

,author,faculty,id,isbn,location,name,publisher,topic,year,type
0,Project management Institute,ISE,1,None,None,A guide to the project management body of know...,"Project Management Institute, Inc.",Industrial Systems Engineering,2008,database
1,"Igor A. Karnovsky, Olga Lebed",CE,2,None,None,Advanced Methods of Structural Analysis,Springer,Civil Engineering,2010,database
2,Alan L. Harvey,BT,3,None,None,Advances in Drug Discovery Techniques,John Wiley & Sons,Biotechnology,1998,database
3,Frank K. Reilly,BAFN,4,None,None,Analysis of investment & management of portfol...,South-Western Cengage Learning,"Finance, Business Administration",2012,database
4,"John S. Lucas, Paul C. Southgate",BT,5,None,None,Aquaculture: Farming Aquatic Animals and Plant...,John Wiley & Sons,Aquatic Resources Management,2011,database
5,Francis D.K. Ching,CE,6,None,None,"Architecture: Form, space and order 3rd ed.",John Wiley & Sons,Civil Engineering,2007,database
6,"Douglas F. Prawitt, Steven M. Glover, William ...",BAFN,7,None,None,Auditing and Assurance Services: A Systematic ...,McGraw Hill,"Finance, Business Administration",2012,database
7,"Ana M. Aransay, Michael Hackenberg, Naiara Rod...",BT,8,None,None,Bioinformatics for High Throughput Sequencing,Springer,Biotechnology,2012,database
8,John L. Semmlow,BME,9,None,None,"Biosignal and Biomedical Image Processing, 2nd...",CRC Press,Biomedical Engineering,2009,database
9,Kenneth D. Weiss,BA,10,None,None,"Building an Import / Export Business, 4th ed.",Wiley &​ Sons,Business Administration,2008,database


In [3]:
csv_book_list_df = pd.read_csv('BookData.csv',names=['id','name','author','pricing','pub_year','publisher','topic','type','quantility'], header=None)
csv_book_list_df['faculty'] = ''
csv_book_list_df['type'] = 'csv'
csv_book_list_df['id'] = csv_book_list_df['id'] + db_book_list.id.count() - 1
csv_book_list_df.set_index('id', drop=False, inplace=True)
csv_book_list_df

,id,name,author,pricing,pub_year,publisher,topic,type,quantility,faculty
id,,,,,,,,,,
165,165,10 real SATs,Unknown,6574,2000,New York : College Entrance Examination Board,Social Sciences,csv,2,
166,166,100 contemporary architects: drawings And sket...,"Lacy, Bill",57886,1991,London : Thames and Hudson,Civil Engineering,csv,1,
167,167,100 desktop publishing exercises,"Youth, Helen",56530,1990,New Jersey : Prentice Hall,Biomedical Engineering,csv,2,
168,168,1001 internet tips,"Malone, Bill",1998,2000,"Houston, Tex. : Jamsa Press",Computer Science And Engineering,csv,1,
169,169,1001 questions answered about trees,"Platt, Rutherford Hayes, 1894-",11902,1992,New York : Dover Publications,Biotechnology,csv,1,
170,170,1001 ways to reward employees,"Nelson, Bod",8328,1994,New York : Workman Publishing,Social Sciences,csv,1,
171,171,1001 Word 2000 tips,"Wyatt, Allen L.",5532,1999,"Houston, TX : Jamsa Press",Business Administration,csv,1,
172,172,101 real money questions : the African America...,"Brown, Jesse B.",3098,2003,New York : John Wiley And Sons,Business Administration,csv,1,
173,173,106 mortgage secrets all borrowers must learn ...,"Eldred, Gary W.",3099,2008,N.J. : John Wiley And Sons,Business Administration,csv,1,


In [4]:
book_list_df = pd.concat([db_book_list, csv_book_list_df])
book_list_df

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,author,faculty,id,isbn,location,name,pricing,pub_year,publisher,quantility,topic,type,year
0,Project management Institute,ISE,1,None,None,A guide to the project management body of know...,NaN,NaN,"Project Management Institute, Inc.",NaN,Industrial Systems Engineering,database,2008
1,"Igor A. Karnovsky, Olga Lebed",CE,2,None,None,Advanced Methods of Structural Analysis,NaN,NaN,Springer,NaN,Civil Engineering,database,2010
2,Alan L. Harvey,BT,3,None,None,Advances in Drug Discovery Techniques,NaN,NaN,John Wiley & Sons,NaN,Biotechnology,database,1998
3,Frank K. Reilly,BAFN,4,None,None,Analysis of investment & management of portfol...,NaN,NaN,South-Western Cengage Learning,NaN,"Finance, Business Administration",database,2012
4,"John S. Lucas, Paul C. Southgate",BT,5,None,None,Aquaculture: Farming Aquatic Animals and Plant...,NaN,NaN,John Wiley & Sons,NaN,Aquatic Resources Management,database,2011
5,Francis D.K. Ching,CE,6,None,None,"Architecture: Form, space and order 3rd ed.",NaN,NaN,John Wiley & Sons,NaN,Civil Engineering,database,2007
6,"Douglas F. Prawitt, Steven M. Glover, William ...",BAFN,7,None,None,Auditing and Assurance Services: A Systematic ...,NaN,NaN,McGraw Hill,NaN,"Finance, Business Administration",database,2012
7,"Ana M. Aransay, Michael Hackenberg, Naiara Rod...",BT,8,None,None,Bioinformatics for High Throughput Sequencing,NaN,NaN,Springer,NaN,Biotechnology,database,2012
8,John L. Semmlow,BME,9,None,None,"Biosignal and Biomedical Image Processing, 2nd...",NaN,NaN,CRC Press,NaN,Biomedical Engineering,database,2009
9,Kenneth D. Weiss,BA,10,None,None,"Building an Import / Export Business, 4th ed.",NaN,NaN,Wiley &​ Sons,NaN,Business Administration,database,2008


In [5]:
def clean_data(x):
    if isinstance(x, str):
        x = str.lower(x.replace(" ", ""))
        x = x.replace("unknown", " ")
        return x.replace(",", " ")
    else:
        return ''

In [6]:
# Apply clean_data function to your features.
features = ['author', 'faculty','publisher','topic']

for feature in features:
    book_list_df[feature+'_clean'] = book_list_df[feature].apply(clean_data)

In [7]:
def create_soup(x):
    temp = ''
    for feature in features:
        temp += x[feature+'_clean'] + ' '
    temp += str.lower(x['name'])
    return temp
#     return x['author'] + ' ' + x['faculty'] + ' ' + x['publisher'] + ' ' +x['name'] + ' ' +x['topic']

In [8]:
book_list_df['soup'] = book_list_df.apply(create_soup, axis=1)
book_list_df

,author,faculty,id,isbn,location,name,pricing,pub_year,publisher,quantility,topic,type,year,author_clean,faculty_clean,publisher_clean,topic_clean,soup
0,Project management Institute,ISE,1,None,None,A guide to the project management body of know...,NaN,NaN,"Project Management Institute, Inc.",NaN,Industrial Systems Engineering,database,2008,projectmanagementinstitute,ise,projectmanagementinstitute inc.,industrialsystemsengineering,projectmanagementinstitute ise projectmanageme...
1,"Igor A. Karnovsky, Olga Lebed",CE,2,None,None,Advanced Methods of Structural Analysis,NaN,NaN,Springer,NaN,Civil Engineering,database,2010,igora.karnovsky olgalebed,ce,springer,civilengineering,igora.karnovsky olgalebed ce springer civileng...
2,Alan L. Harvey,BT,3,None,None,Advances in Drug Discovery Techniques,NaN,NaN,John Wiley & Sons,NaN,Biotechnology,database,1998,alanl.harvey,bt,johnwiley&sons,biotechnology,alanl.harvey bt johnwiley&sons biotechnology a...
3,Frank K. Reilly,BAFN,4,None,None,Analysis of investment & management of portfol...,NaN,NaN,South-Western Cengage Learning,NaN,"Finance, Business Administration",database,2012,frankk.reilly,bafn,south-westerncengagelearning,finance businessadministration,frankk.reilly bafn south-westerncengagelearnin...
4,"John S. Lucas, Paul C. Southgate",BT,5,None,None,Aquaculture: Farming Aquatic Animals and Plant...,NaN,NaN,John Wiley & Sons,NaN,Aquatic Resources Management,database,2011,johns.lucas paulc.southgate,bt,johnwiley&sons,aquaticresourcesmanagement,johns.lucas paulc.southgate bt johnwiley&sons ...
5,Francis D.K. Ching,CE,6,None,None,"Architecture: Form, space and order 3rd ed.",NaN,NaN,John Wiley & Sons,NaN,Civil Engineering,database,2007,francisd.k.ching,ce,johnwiley&sons,civilengineering,francisd.k.ching ce johnwiley&sons civilengine...
6,"Douglas F. Prawitt, Steven M. Glover, William ...",BAFN,7,None,None,Auditing and Assurance Services: A Systematic ...,NaN,NaN,McGraw Hill,NaN,"Finance, Business Administration",database,2012,douglasf.prawitt stevenm.glover williamf.messier,bafn,mcgrawhill,finance businessadministration,douglasf.prawitt stevenm.glover williamf.messi...
7,"Ana M. Aransay, Michael Hackenberg, Naiara Rod...",BT,8,None,None,Bioinformatics for High Throughput Sequencing,NaN,NaN,Springer,NaN,Biotechnology,database,2012,anam.aransay michaelhackenberg naiararodríguez...,bt,springer,biotechnology,anam.aransay michaelhackenberg naiararodríguez...
8,John L. Semmlow,BME,9,None,None,"Biosignal and Biomedical Image Processing, 2nd...",NaN,NaN,CRC Press,NaN,Biomedical Engineering,database,2009,johnl.semmlow,bme,crcpress,biomedicalengineering,johnl.semmlow bme crcpress biomedicalengineeri...
9,Kenneth D. Weiss,BA,10,None,None,"Building an Import / Export Business, 4th ed.",NaN,NaN,Wiley &​ Sons,NaN,Business Administration,database,2008,kennethd.weiss,ba,wiley&​sons,businessadministration,kennethd.weiss ba wiley&​sons businessadminist...


In [9]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(book_list_df['soup'])
count_matrix

<1125x4116 sparse matrix of type '<class 'numpy.int64'>'
	with 11078 stored elements in Compressed Sparse Row format>

In [10]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
for i in range(cosine_sim2.shape[0]):
    print(i,':',cosine_sim2[i],"\n")

0 : [1. 0. 0. ... 0. 0. 0.] 

1 : [0. 1. 0. ... 0. 0. 0.] 

2 : [0. 0. 1. ... 0. 0. 0.] 

3 : [0.15384615 0.0877058  0.         ... 0.         0.         0.        ] 

4 : [0.07161149 0.         0.24494897 ... 0.         0.         0.        ] 

5 : [0.08006408 0.18257419 0.18257419 ... 0.         0.         0.        ] 

6 : [0.06726728 0.         0.         ... 0.         0.         0.        ] 

7 : [0.         0.09128709 0.18257419 ... 0.         0.         0.        ] 

8 : [0.0836242 0.        0.        ... 0.        0.        0.       ] 

9 : [0.16012815 0.         0.09128709 ... 0.         0.         0.        ] 

10 : [0.13453456 0.0766965  0.         ... 0.         0.         0.        ] 

11 : [0. 0. 0. ... 0. 0. 0.] 

12 : [0.14322297 0.         0.         ... 0.         0.         0.        ] 

13 : [0.0836242  0.09534626 0.         ... 0.         0.         0.        ] 

14 : [0.10482848 0.         0.11952286 ... 0.         0.         0.        ] 

15 : [0.         0.    

481 : [0.         0.09534626 0.         ... 0.         0.         0.        ] 

482 : [0.0877058 0.1       0.        ... 0.        0.        0.       ] 

483 : [0.         0.09128709 0.         ... 0.         0.         0.        ] 

484 : [0.         0.09534626 0.         ... 0.         0.         0.        ] 

485 : [0.  0.1 0.  ... 0.  0.  0. ] 

486 : [0.         0.10540926 0.         ... 0.         0.         0.        ] 

487 : [0.  0.1 0.  ... 0.  0.  0. ] 

488 : [0.         0.11952286 0.         ... 0.         0.         0.11952286] 

489 : [0.        0.0745356 0.0745356 ... 0.        0.        0.       ] 

490 : [0.         0.08164966 0.         ... 0.         0.         0.        ] 

491 : [0.         0.08164966 0.         ... 0.         0.         0.08164966] 

492 : [0.         0.10540926 0.10540926 ... 0.         0.         0.        ] 

493 : [0.         0.07905694 0.         ... 0.         0.08838835 0.07905694] 

494 : [0.        0.1118034 0.        ... 0.        0.   

716 : [0. 0. 0. ... 0. 0. 0.] 

717 : [0.         0.09128709 0.         ... 0.         0.         0.        ] 

718 : [0.         0.09534626 0.         ... 0.         0.         0.        ] 

719 : [0. 0. 0. ... 0. 0. 0.] 

720 : [0.09805807 0.         0.1118034  ... 0.         0.         0.        ] 

721 : [0.         0.         0.10540926 ... 0.         0.         0.        ] 

722 : [0.         0.         0.18257419 ... 0.         0.         0.        ] 

723 : [0.         0.         0.23904572 ... 0.         0.         0.        ] 

724 : [0.         0.10540926 0.         ... 0.         0.         0.        ] 

725 : [0.         0.11952286 0.         ... 0.         0.         0.        ] 

726 : [0. 0. 0. ... 0. 0. 0.] 

727 : [0. 0. 0. ... 0. 0. 0.] 

728 : [0. 0. 0. ... 0. 0. 0.] 

729 : [0. 0. 0. ... 0. 0. 0.] 

730 : [0. 0. 0. ... 0. 0. 0.] 

731 : [0.         0.         0.21081851 ... 0.         0.         0.        ] 

732 : [0. 0. 0. ... 0. 0. 0.] 

733 : [0. 0. 0. ... 0. 0

In [15]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title_id, cosine_sim):
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[title_id]))
    print (sim_scores)

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    title_indices = [i[0] for i in sim_scores]
    print(title_indices)
    return title_indices
    # Return the top 10 most similar movies
#     return book_list_df['name'].iloc[title_indices]

In [16]:
# for i in range(len(book_list_df)):
print(book_list_df.iloc[2],'\n')
print(get_recommendations(2, cosine_sim2),'\n')

author                                                Alan L. Harvey
faculty                                                           BT
id                                                                 3
isbn                                                            None
location                                                        None
name                           Advances in Drug Discovery Techniques
pricing                                                          NaN
pub_year                                                         NaN
publisher                                          John Wiley & Sons
quantility                                                       NaN
topic                                                  Biotechnology
type                                                        database
year                                                            1998
author_clean                                            alanl.harvey
faculty_clean                     

In [ ]:
similarity_matrix_item = pd.DataFrame(list(Similarity.objects.filter(id).values()))
# Change transaction_master_db to data frame
transaction_master_df = pd.DataFrame(list(Master.objects.all().values()))